In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
classes = np.array([str(l) for l in range(28)])
df = pd.read_csv('aug_train.csv'); len(df)
PATH = Path('./')
SAMPLE = Path('sample_submission.csv')
sample_sub = pd.read_csv(PATH/SAMPLE)
test_names = list(sample_sub.Id.values)

In [3]:
def count_classes(df):
    out = np.zeros(28, dtype=int)
    for i in range(len(df)):
        idxs = [int(x) for x in df.Target.iloc[i].split()]
        out[idxs]+=1
    return out

In [4]:
cc = count_classes(df)

In [5]:
hard_classes = np.arange(28)[cc<500]; hard_classes

array([ 8,  9, 10, 15, 17, 20, 27])

# Model ensemble

In [7]:
model_name = 'ensemble'

In [73]:
r18 = np.load('predsR18.npy')
r18tta = np.load('preds_tta_r18_ths.npy')

In [15]:
r34 = np.load('predsR34.npy')
r34tta = np.load('preds_tta_r34_ths.npy')

In [16]:
r18ths = np.load('r18_ths.npy')

In [17]:
r34ths = np.load('r34_ths.npy')

In [19]:
r18ths, r34ths

(array([0.495, 0.37 , 0.45 , 0.38 , 0.42 , 0.39 , 0.34 , 0.43 , 0.385,
        0.45 , 0.42 , 0.32 , 0.34 , 0.345, 0.395, 0.515, 0.335, 0.43 ,
        0.335, 0.41 , 0.45 , 0.43 , 0.365, 0.465, 0.41 , 0.475, 0.35 ,
        0.3  ]),
 array([0.485, 0.43 , 0.42 , 0.41 , 0.465, 0.455, 0.43 , 0.425, 0.685,
        0.66 , 0.75 , 0.46 , 0.45 , 0.415, 0.475, 0.39 , 0.445, 0.54 ,
        0.44 , 0.405, 0.58 , 0.43 , 0.44 , 0.415, 0.485, 0.44 , 0.47 ,
        0.685]))

In [119]:
ense_ref = (0.4*r18 + 0.6*r34)
ense = (r18tta+r34tta)/2

In [120]:
def preds2frame(preds, th):
    print(preds.shape)
    res = np.array([" ".join(classes[(np.where(pp>th))])for pp in preds])
    return pd.DataFrame(np.array([test_names, res]).T, columns = ['Id','Predicted'])

In [121]:
df_r18 = preds2frame(r18, 0.4)
df_r34 = preds2frame(r34, 0.4)

df_r18tta = preds2frame(r18tta, 0.4)
df_r34tta = preds2frame(r34tta, 0.4)

df_ensemble_ref = preds2frame(ense_ref, 0.4)
df_ensemble = preds2frame(ense, 0.4)

(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)


In [122]:
df = pd.concat([df_r18,
                df_r18tta['Predicted'],
                df_r34['Predicted'],
                df_r34tta['Predicted'], 
                df_ensemble_ref['Predicted'],
                df_ensemble['Predicted']],ignore_index=True, axis=1)
df.columns = ['Id', 'r18', 'r18tta', 'r34' , 'r34tta', 'ense_ref', 'ense']

idxs = np.where(df_ensemble_ref['Predicted']!=df_ensemble['Predicted'])

print(f'Different ensembles : {len(df.iloc[idxs])}')
df.iloc[idxs]

Different ensembles : 3189


,Id,r18,r18tta,r34,r34tta,ense_ref,ense
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25,5 25,5,5,5,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25,0 5 25,0 5 21 25,0 5 21 25,0 5 25,0 5 21 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25,0 25,0 16 18 25,0 25,0 16 25,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7,0 25,0,,0,0 25
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 4 23 25,0 4 23 25,4 5 25,4 25,0 4 25,4 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25,0 25,0 6,0,0,0 25
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 2 25,2 21 25,2 4 21 25,2 4 21 26,2 21 25,2 4 21
10,002729d2-bace-11e8-b2b8-ac1f6b6435d0,25,0 25,18,18,25,18
14,00407c16-bad3-11e8-b2b8-ac1f6b6435d0,0 25,0 25,,,,25
15,005ce2ea-bacc-11e8-b2b8-ac1f6b6435d0,25,25,18 25,18,18 25,


In [123]:
idxs = np.where(df_r18.Predicted != df_r18tta.Predicted)

In [ ]:
len(idxs[0])

In [116]:
idxs = np.where(df_r34.Predicted != df_r34tta.Predicted)

In [117]:
len(idxs[0])

3746

In [118]:
df_ensemble_ref.to_csv(f'{model_name}_ref.csv', index=False)

In [34]:
df_r34tta.to_csv('r34_ths_tta.csv', index=False)